In [160]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [161]:
# Celda 1 - Importaciones y configuración inicial
"""
Recolector de Imágenes para Dataset de Lenguaje de Señas
------------------------------------------------------
Este script captura imágenes desde la cámara web para crear un dataset de entrenamiento
para el clasificador de lenguaje de señas.
"""

# === IMPORTACIÓN DE BIBLIOTECAS ===
import os
import cv2
import time
from pathlib import Path
from google.colab import files  # Necesario para Colab
from IPython.display import display, Javascript, clear_output
from google.colab.patches import cv2_imshow  # Reemplaza cv2.imshow en Colab
from google.colab.output import eval_js
from base64 import b64decode, b64encode

In [162]:
# === CONFIGURACIÓN INICIAL ===
os.chdir('/content/drive/MyDrive/Cursos/Computer_vision/Hand_Gesture_detection/code/')

AUTOR = 'Sergio' # Natalia, Doro y Fergus
DATA_DIR = os.path.join('./data/', AUTOR)  # Combina './data/' con el nombre del autor
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Parámetros de configuración
number_of_classes = 3  # Número de clases (alfabeto)
dataset_size = 100  # Número de imágenes por clase
capture_interval = 0.1  # Intervalo de captura en segundos

In [163]:
def create_class_directories():
    """Crear directorios para cada clase"""
    for class_num in range(number_of_classes):
        class_dir = os.path.join(DATA_DIR, str(class_num))
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
            print(f'Creado directorio para clase {class_num}: {class_dir}')

In [182]:
def capturar_imagen2(dataset_size, capture_interval):  # Agregamos capture_interval como parámetro
    # Primero limpiamos cualquier instancia anterior

    js = Javascript(f'''
    async function captureImagesImproved() {{
        // Crear contenedor principal con estilo
        const div = document.createElement('div');
        div.setAttribute('data-camera-container', 'true');
        div.style.textAlign = 'center';
        div.style.padding = '20px';
        div.style.backgroundColor = '#f5f5f5';
        div.style.borderRadius = '10px';
        div.style.margin = '20px';

        // Verificar si ya existe una instancia
        if (document.querySelector('[data-camera-container]')) {{
            return null;
        }}

        const startButton = document.createElement('button');
        startButton.textContent = 'Iniciar Cámara';
        startButton.style.margin = '10px';
        startButton.style.padding = '10px 20px';
        startButton.style.backgroundColor = '#4CAF50';
        startButton.style.color = 'white';
        startButton.style.border = 'none';
        startButton.style.borderRadius = '5px';
        startButton.style.cursor = 'pointer';

        const captureButton = document.createElement('button');
        captureButton.textContent = 'Comenzar Captura';
        captureButton.style.margin = '10px';
        captureButton.style.padding = '10px 20px';
        captureButton.style.backgroundColor = '#2196F3';
        captureButton.style.color = 'white';
        captureButton.style.border = 'none';
        captureButton.style.borderRadius = '5px';
        captureButton.style.cursor = 'pointer';
        captureButton.style.display = 'none';

        const video = document.createElement('video');
        video.style.display = 'none';
        video.style.maxWidth = '100%';
        video.style.borderRadius = '5px';
        video.style.margin = '10px 0';

        div.appendChild(startButton);
        div.appendChild(captureButton);
        div.appendChild(video);
        document.body.appendChild(div);

        let stream;
        try {{
            return await new Promise(async (resolve, reject) => {{
                let isCapturing = false;  // Flag para evitar múltiples capturas

                startButton.onclick = async () => {{
                    if (isCapturing) return;
                    try {{
                        stream = await navigator.mediaDevices.getUserMedia({{video: true}});
                        video.srcObject = stream;
                        video.style.display = 'block';
                        await video.play();
                        startButton.style.display = 'none';
                        captureButton.style.display = 'inline-block';
                    }} catch (error) {{
                        console.error('Error accessing camera:', error);
                        alert('Error accessing camera. Please grant permission or check if another application is using it.');
                        reject(error);
                    }}
                }};

                captureButton.onclick = async () => {{
                    if (isCapturing) return;
                    isCapturing = true;
                    try {{
                        const canvas = document.createElement('canvas');
                        canvas.width = video.videoWidth;
                        canvas.height = video.videoHeight;
                        const context = canvas.getContext('2d');
                        const images = [];

                        const progress = document.createElement('div');
                        progress.style.margin = '10px';
                        progress.style.color = 'black';
                        progress.style.fontWeight = 'bold';
                        div.appendChild(progress);

                        for(let i = 0; i < {dataset_size}; i++) {{
                            context.drawImage(video, 0, 0);
                            images.push(canvas.toDataURL('image/jpeg', 0.8));
                            progress.textContent = `Capturando imagen ${{i + 1}} de {dataset_size}`;
                            await new Promise(resolve => setTimeout(resolve, {capture_interval * 1000}));
                        }}

                        if (stream) {{
                            stream.getTracks().forEach(track => track.stop());
                        }}
                        div.remove();
                        resolve(images);
                    }} catch (error) {{
                        console.error('Error during capture:', error);
                        if (stream) {{
                            stream.getTracks().forEach(track => track.stop());
                        }}
                        div.remove();
                        reject(error);
                    }} finally {{
                        isCapturing = false;
                    }}
                }};
            }});
        }} catch (error) {{
            console.error('Error:', error);
            if (stream) {{
                stream.getTracks().forEach(track => track.stop());
            }}
            div.remove();
            return null;
        }}
    }}

    ''')

    display(js)

    # Esperamos y retornamos el resultado
    return eval_js('captureImagesImproved()')

In [177]:
def capturar_imagen(dataset_size):
  js = Javascript('''
        async function captureImages(dataset_size) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Comenzar captura';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';

            try {
                const stream = await navigator.mediaDevices.getUserMedia({video: true});
                document.body.appendChild(div);
                div.appendChild(video);
                video.srcObject = stream;
                await video.play();

                await new Promise((resolve) => capture.onclick = resolve);

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                const context = canvas.getContext('2d');

                const images = [];

                for(let i = 0; i < ''' + str(dataset_size) + '''; i++) {
                    context.drawImage(video, 0, 0);
                    images.push(canvas.toDataURL('image/jpeg', 0.8));
                    await new Promise(resolve => setTimeout(resolve, ''' + str(capture_interval * 1000) + '''));
                }

                stream.getTracks().forEach(track => track.stop());
                div.remove();

                return images;
            } catch (error) {
                console.error('Error:', error);
                return null;
            }
        }
        ''')
  display(js)
  return eval_js('captureImages()')

In [178]:
# Función principal de captura
def capturar_imagenes(class_num, dataset_size,capture_interval):
    # Asegurar que existe el directorio para esta clase
    class_dir = os.path.join(DATA_DIR, str(class_num))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    print(f'Preparando captura para clase {class_num}')
    print(f'Las imágenes se guardarán en: {class_dir}')
    print('Presiona "Comenzar captura" cuando estés listo')

    try:
        images_data = capturar_imagen2(dataset_size,capture_interval)
        if images_data is None:
            raise Exception("No se pudieron capturar las imágenes")

        # Procesar y guardar las imágenes en el directorio de la clase correspondiente
        for i, img_data in enumerate(images_data):
            try:
                binary = b64decode(img_data.split(',')[1])
                save_path = os.path.join(class_dir, f'{i}.jpg')
                with open(save_path, 'wb') as f:
                    f.write(binary)
                print(f'Clase {class_num}: Guardada imagen {i+1} de {dataset_size} en {save_path}')
            except Exception as e:
                print(f'Error guardando imagen {i} para clase {class_num}: {str(e)}')

        return True

    except Exception as e:
        print(f'Error en la captura para clase {class_num}: {str(e)}')
        return False

    print('\n¡Recolección de datos completada!')
    print(f'Las imágenes se guardaron en: {DATA_DIR}')

In [183]:
# Ejecutar la captura
create_class_directories()

# Capturar imágenes para cada clase
for class_num in range(number_of_classes):
    print(f'\nIniciando captura para la clase {class_num}')
    print(f'Se capturarán {dataset_size} imágenes')

    success = False
    while not success:
        print(dataset_size)
        success = capturar_imagenes(class_num, dataset_size,capture_interval)
        if not success:
            input(f"Error en la captura de la clase {class_num}. Presiona Enter para intentar de nuevo...")
        clear_output(wait=True)

    print(f'Completada la captura para la clase {class_num}')
    # Verificar número de imágenes guardadas
    class_dir = os.path.join(DATA_DIR, str(class_num))
    num_images = len([f for f in os.listdir(class_dir) if f.endswith('.jpg')])
    print(f'Imágenes guardadas en clase {class_num}: {num_images}')
    time.sleep(2)

# Verificación final
print('\nVerificación final de datos guardados:')
for class_num in range(number_of_classes):
    class_dir = os.path.join(DATA_DIR, str(class_num))
    num_images = len([f for f in os.listdir(class_dir) if f.endswith('.jpg')])
    print(f'Clase {class_num}: {num_images} imágenes')

print('\n¡Recolección de datos completada!')

Completada la captura para la clase 2
Imágenes guardadas en clase 2: 100

Verificación final de datos guardados:
Clase 0: 100 imágenes
Clase 1: 100 imágenes
Clase 2: 100 imágenes

¡Recolección de datos completada!
